In [5]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier

s3 = boto3.resource('s3')
bucket_name = 'data-445-timlincoln'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [6]:
# Dropping N/A variables
heart = heart.dropna()

In [7]:
# Defining the input and target variables
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'sysBP', 'glucose']]
Y = heart['TenYearCHD']

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [10]:
# Scaling the data
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [9]:
# Identify important variables with Lasso
lasso_md = LassoCV().fit(X_train, Y_train)

# Extracting best lambda
lasso_lambda = lasso_md.alpha_

# Fitting lasso with best lambda
lasso_md = Lasso(alpha = lasso_lambda).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.0772079 ,  0.24007847,  0.03891451,  0.10433484,  0.01823936,
       -0.0442922 ,  0.57525707,  0.36569781])

In [ ]:
# Defining
X_train_logit = X_train[['male', 'age', 'totChol', 'sysBP', 'glucose']]
X_test_logit = X_test[['male', 'age', 'totChol', 'sysBP', 'glucose']]

# Building
logit_md = LogisticRegression().fit(X_train_logit, Y_train)

# Predicting
logit_pred = logit_md.predict_proba(X_test_logit)[:, 1]

# Changning
logit_label = np.where(logit_pred < 0.1, 0, 1)

# Computing
recall_score(Y_test, logit_label)